In [ ]:
# 2021.08.12
# kakaoMap_crawling_4.ipynb 수정버전
# 수정사항 : 리뷰 내용 더보기

In [4]:
import os

from time import sleep
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from bs4 import BeautifulSoup

import pymysql as my
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore')

##########################################################################
##################### variable related selenium ##########################
##########################################################################
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('lang=ko_KR')


driver = webdriver.Chrome('./webdriver/chromedriver')



##### 가게 정보를 담는 클레스
class STORE_Info:
    place_code = ''
    place_name = ''
    place_address = ''
    place_local  = ''
    place_category  = ''
    
    # 생성자
    def __init__(self, store_code = None, place_name = None, place_address = None, place_local = None, place_category = None):
        self.store_code = store_code
        self.place_name =  place_name
        self.place_address = place_address
        self.place_local  = place_local
        self.place_category  = place_category
        
    # 데이터 확인
    def data_check(self):
        print('가게코드 : %s, 가계명 : %s, 주소 : %s, 지역 : %s, 종류 : %s' %(self.store_code, self.place_name, self.place_address, 
                                                                 self.place_local, self.place_category))
        
        
##### 리뷰 정보를 담는 클레스
class REVIEW_Info:
    review_code = ''
    place_code = ''
    comment = ''
    rating = ''
    user_id  = ''
    timestamp  = ''

    
    # 생성자
    def __init__(self, review_code=None, store_code =None ,comment = None, rating = None, user_id = None, timestamp = None):
        self.review_code = review_code
        self.store_code = store_code
        self.comment =  comment
        self.rating = rating
        self.user_id  = user_id
        self.timestamp  = timestamp 

    # 데이터 확인
    def data_check(self):
        print('리뷰코드 : %s, 가게코드 : %s, ID : %s, 리뷰 : %s, 평점 : %s, 날짜 : %s' %(self.review_code, self.store_code, 
                                                                           self.user_id,self.comment, self.rating, self.timestamp))

        
### DB 다루는 클래스
class DBHelper:
    '''
    맴버변수 : 커넥션 
    '''
    conn = None
    '''
    생성자
    '''
    def __init__(self):
        self.db_init()
    '''
    맴버 함수
    '''
#     def db_init(self):
#         self.conn = my.connect(
#                         host='3.34.111.86',
#                         user='root',
#                         password='123456',
#                         db='project_db',
#                         charset='utf8mb4',
#                         cursorclass=my.cursors.DictCursor
#         )
        
    def db_init(self):
        self.conn = my.connect(
                        host='127.0.0.1',
                        user='root',
                        password='960525',
                        db='project_db',
                        charset='utf8mb4',
                        cursorclass=my.cursors.DictCursor
        )

        


    def db_free(self):
        if self.conn:
            self.conn.close()
            
    def db_selectKeyword_STORE(self):
        # 커서 오픈
        # with => 닫기를 처리를 자동으로 처리해준다 => I/O 많이 사용
        rows = None
        with self.conn.cursor() as cursor:
            sql  = "select * from STORE;"
            cursor.execute(sql)
            rows = cursor.fetchall()
        return rows           
    
    def db_selectKeyword_REVIEW(self):
        # 커서 오픈
        # with => 닫기를 처리를 자동으로 처리해준다 => I/O 많이 사용
        rows = None
        with self.conn.cursor() as cursor:
            sql  = "select * from REVIEW;"
            cursor.execute(sql)
            rows = cursor.fetchall()
        return rows      

    def db_insertCrawlingData_STORE(self, store_code, place_name, place_address, place_local, place_category):
        with self.conn.cursor() as cursor:
            sql = "INSERT INTO STORE (STORE_CODE, STORE_NAME, ADDRESS, LOCAL_CODE, CATRGORY_CODE) VALUES( %s, %s, %s, %s, %s )"
            cursor.execute(sql, (store_code, place_name, place_address, place_local, place_category)) 
        self.conn.commit()
        
    
    def db_insertCrawlingData_REVIEW(self, review_code, store_code, user_id, rating, comment, timestamp):
        with self.conn.cursor() as cursor:
            sql = "INSERT INTO REVIEW (REVIEW_CODE, STORE_CODE,  USER_ID, SCORE, REVIEW, UPLOAD_DATE) VALUES( %s, %s, %s, %s, %s, %s )"
            cursor.execute(sql, (review_code, store_code, user_id, rating, comment, timestamp))
        self.conn.commit()
    


rating_df = pd.DataFrame()
restaurant_df = pd.DataFrame()
db = DBHelper()


        
        
##### 메인 코드 #####
def main():
    global driver, load_wb, review_num, local, number_local, number_store

    driver.implicitly_wait(4)  # 렌더링 될때까지 기다린다 4초
    driver.get('https://map.kakao.com/')  # 주소 가져오기

     #### 검색할 목록
#     place_infos = ['종로구 맛집', '중구 맛집', ' 용산구 맛집', '성동구 맛집', '광진구 맛집', '동대문구 맛집', '중랑구 맛집', '성북구 맛집', '강북구 맛집']
#     place_infos = ['도봉구 맛집', '노원구 맛집', '은평구 맛집', '서대문구 맛집', '마포구 맛집', '양천구 맛집', '강서구 맛집','구로구 맛집']
#      place_infos = ['금천구 맛집', '영등포구 맛집', '동작구 맛집', '관악구 맛집', '서초구 맛집', '강남구 맛집', '송파구 맛집', '강동구 맛집']
    
    place_infos = ['금천구 맛집', '영등포구 맛집', '동작구 맛집', '관악구 맛집', '서초구 맛집', '강남구 맛집', '송파구 맛집', '강동구 맛집']

    for i, place in enumerate(place_infos):
        print("##### {0} : {1}".format(i,place))
        number_local = 0
        number_store = 0
        
        local = read_local_code(place)
        search(place, local)
        create_csv_file(place)

    driver.quit()
    print("finish")

#     db.db_selectKeyword_STORE()
#     db.db_selectKeyword_REVIEW()
    


#### 가게 정보 찾기
def search(place, local):
    """
    검색한 페이지 음식점 정보 크롤링 하는 함수
    :param place: 리뷰 정보 찾을 장소이름
    :param local : 지역구 코드
    """
    global driver

    search_area = driver.find_element_by_xpath('//*[@id="search.keyword.query"]')  # 검색 창
    search_area.send_keys(place)  # 검색어 입력
    driver.find_element_by_xpath('//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)  # Enter로 검색
    driver.find_element_by_xpath('//*[@id="info.search.place.more"]').send_keys(Keys.ENTER) # 더보기
    
    xPath = '//*[@id="info.search.page.no1"]'
    driver.find_element_by_xpath(xPath).send_keys(Keys.ENTER) # 첫번째 페이지로 이동
    sleep(1)

    # 검색된 정보가 있는 경우에만 탐색
    # 1번 페이지 place list 읽기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    place_lists = soup.select('.placelist > .PlaceItem') # 검색된 장소 목록

    # 검색된 첫 페이지 장소 목록 크롤링하기
    crawling(place, local, place_lists)
    for i in range(2, 6):
        # 페이지 넘기기
        xPath = '//*[@id="info.search.page.no' + str(i) + '"]'
        driver.find_element_by_xpath(xPath).send_keys(Keys.ENTER)
        sleep(1)

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        place_lists = soup.select('.placelist > .PlaceItem') # 장소 목록 list
                
        crawling(place, local, place_lists)
                
        # 다음 페이지 넘기기
        if i==5:
            driver.find_element_by_xpath('//*[@id="info.search.page.next"]').send_keys(Keys.ENTER)

    
    # 전체 페이지
    while True:
        try:
#            2~ 5페이지 읽기
            for i in range(1, 6):
                # 페이지 넘기기
                xPath = '//*[@id="info.search.page.no' + str(i) + '"]'
                driver.find_element_by_xpath(xPath).send_keys(Keys.ENTER)
                sleep(1)

                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                place_lists = soup.select('.placelist > .PlaceItem') # 장소 목록 list
                
                crawling(place, local, place_lists)
                
                # 다음 페이지 넘기기
                if i==5:
                    driver.find_element_by_xpath('//*[@id="info.search.page.next"]').send_keys(Keys.ENTER)

        except ElementNotInteractableException:
            print('end page')
            break
            
        finally:
            search_area = driver.find_element_by_xpath('//*[@id="search.keyword.query"]')
            search_area.clear()


#### 가게 정보 크롤링 하기
def crawling(place, local, place_lists):
    """
    페이지 목록을 받아서 크롤링 하는 함수
    :param place: 리뷰 정보 찾을 장소이름
    :param local : 지역구 코드
    :param place_lists : 페이지 내에 있는 음식점 목록
    """
    
    global restaurant_df, number_store, db

    
    while_flag = False
    for i, place in enumerate(place_lists):
        place_name = place.select('.head_item > .tit_name > .link_name')[0].text  # place name
        place_address = place.select('.info_item > .addr > p')[0].text  # place address
        place_local = place.select('.info_item > .addr > .lot_number')[0].text
        place_category = place.select('.head_item > .subcategory')[0].text
        place_detail = place.select('.info_item > .contact> .moreview')[0].get('href') # place detail
        
        number_store += 1
        store_code = str('STORE_') + local + str(number_store).zfill(5) # 가게 코드 (ex. STORE_GC00001)
        
        store_Info = STORE_Info(store_code, place_name, place_address, place_local, place_category)
        store_Info.data_check()
        
        
        #### DataFrame 저장 ####
        row = {'store_code': store_code, "ItemID":place_name, "address": place_address, "local" : place_local, "category": place_category}
        row = pd.DataFrame(row, index=[1])
        restaurant_df = restaurant_df.append(row, ignore_index=True)
        
        
        #### 데이터 베이스 저장 #####
        db.db_insertCrawlingData_STORE(store_code, place_name, place_address, place_local, place_category)
        

        
        #### 리뷰 크롤링 ####
        driver.execute_script('window.open("about:blank", "_blank");')
        driver.switch_to.window(driver.window_handles[-1])
        driver.get(place_detail) # 상세정보 탭으로 변환
        sleep(1)
        
        
        print('####', place_name)
        sleep(1)
        # 첫 페이지
        extract_review(place_name, store_code , local) # 리뷰 추출

        # 2-5 페이지
        idx = 3
        try:
            page_num = len(driver.find_elements_by_class_name('link_page')) # 페이지 수 찾기
            for i in range(page_num-1):
                # css selector를 이용해 페이지 버튼 누르기
                driver.find_element_by_css_selector('#mArticle > div.cont_evaluation > div.evaluation_review > div > a:nth-child(' + str(idx) +')').send_keys(Keys.ENTER)
                sleep(1)
                extract_review(place_name, store_code, local)
                idx += 1
            driver.find_element_by_link_text('다음').send_keys(Keys.ENTER) # 5페이지가 넘는 경우 다음 버튼 누르기
            
            sleep(1)
            extract_review(place_name, store_code, local) # 리뷰 추출
            
            # 그 이후 페이지
            while True:
                idx = 4
                page_num = len(driver.find_elements_by_class_name('link_page')) #페이지 수 찾기
                for i in range(page_num-1):
                    driver.find_element_by_css_selector('#mArticle > div.cont_evaluation > div.evaluation_review > div > a:nth-child(' + str(idx) +')').send_keys(Keys.ENTER)
                    sleep(1)
                    extract_review(place_name, store_code, local)
                    idx += 1
                driver.find_element_by_link_text('다음').send_keys(Keys.ENTER) # 10페이지 이상으로 넘어가기 위한 다음 버튼 클릭
                sleep(1)
                extract_review(place_name, store_code, local) # 리뷰 추출            
            
        except (NoSuchElementException, ElementNotInteractableException):
            print("no review in crawling")

        driver.close()
        driver.switch_to.window(driver.window_handles[0])  # 검색 탭으로 전환


#### 리뷰 크롤링 하기
def extract_review(place_name, store_code, local):
    """
    리뷰 크롤링 하는 함수
    :param place_place: 리뷰 정보 찾을 장소이름
    :param local : 지역구 코드
    """
    
    global driver, rating_df, number_local, db

    ret = True
    
#     driver.find_element_by_xpath('//*[@id="mArticle"]/div[4]/div[4]/ul/li[4]/div[2]/p/button').send_keys(Keys.ENTER)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 첫 페이지 리뷰 목록 찾기
    review_lists = soup.select('.list_evaluation > li')

    # 리뷰가 있는 경우
    if len(review_lists) != 0:
        for i, review in enumerate(review_lists):
            comment = review.select('.txt_comment > span') # 리뷰
            rating = review.select('.grade_star > em') # 별점
            user_id = review.select('.append_item > a[data-userid]') # user-id 정보 html 파싱
#             user_name = review.select('.append_item > a[data-username]') # user-name 정보 html 파싱
            timestamp = review.select(' div > span.time_write') #시간정보
            number_local += 1
                  

            #### 리뷰 클래스에 정보 담기 ####
            review_info = REVIEW_Info()
            
            review_info.review_code = local + str(number_local).zfill(6) # 리뷰 코드 (ex. GC000001)
            review_info.store_code = store_code
            

            if len(comment) != 0: # 리뷰
                if comment[0].text:
                    review_info.comment = comment[0].text 
                else:
                    review_info.comment = None
            else:
                review_info.comment = None
            
            if len(rating) != 0: # 별점
                review_info.rating = rating[0].text.replace('점', '')
            else:
                review_info.rating = None

            if(len(user_id) != 0): # 유저id
                if(user_id[0].get('data-userid')):
                    review_info.user_id = user_id[0].get('data-userid')
                else:
                    review_info.user_id = None
            else:
                review_info.user_id = None

            if len(timestamp) != 0: # 시간
                review_info.timestamp = timestamp[0].text
            else:
                review_info.timestamp = None
                
            review_info.data_check()
            
            
            
            #### DB 에 정보 담기 ####
            db.db_insertCrawlingData_REVIEW(review_info.review_code, review_info.store_code, review_info.user_id, review_info.rating, review_info.comment, review_info.timestamp)
            
            
            #### DataFrame 에 정보 담기 ####
            try:
                row = {"review_code" : review_info.review_code, "place_code": review_info.store_code, "ItemID": place_name, 
                       "UserID":review_info.user_id, "review" : review_info.comment,
                       "Rating":review_info.rating, "Timestamp":review_info.timestamp}
                row = pd.DataFrame(row, index=[i])
                rating_df = rating_df.append(row, ignore_index=True)
            
            except:
                row = {"review_code" : review_info.review_code, "place_code": store_code, "ItemID":place_name, 
                       "UserID": None, "review" : None, "Rating":None, "Timestamp":review_info.timestamp}
                row = pd.DataFrame(row, index=[i])
                rating_df = rating_df.append(row, ignore_index=True)
                
    else:
        print('no review in extract')
        ret = False

    return ret



#### CSV 파일로 저장 ####
def create_csv_file(place):
    global rating_df, restaurant_df

    rating_df.to_csv('%s_rating_df.csv' %place)
    rating_df.to_csv('%s_rating_df_ko.csv' %place, sep=',', na_rep='NaN', encoding='utf-8-sig')
        
    restaurant_df.to_csv('%s_restaurant_df.csv'  %place)
    restaurant_df.to_csv('%s_restaurant_df_ko.csv'  %place, sep=',', na_rep='NaN', encoding='utf-8-sig')
    
    

#### 지역구 코드 뽑아오기 ####
def read_local_code(place):
    
    local_place = place.split(' ')[0]

    local_code = pd.read_csv('local_code.csv', encoding='utf-8', index_col = 'local')
    return local_code.at[local_place, 'local_code']

#### 메인 ####
if __name__ == "__main__":
    main()

##### 0 : 금천구 맛집
가게코드 : STORE_GC00001, 가계명 : 광고만두라, 주소 : 서울 금천구 시흥대로 97 BB53호, 지역 : (지번) 시흥동 984, 종류 : 한식
#### 광고만두라
리뷰코드 : GC000001, 가게코드 : STORE_GC00001, ID : tghmif, 리뷰 : 드뎌열은듯? 전화가네 , 평점 : 4, 날짜 : 2021.08.12.
리뷰코드 : GC000002, 가게코드 : STORE_GC00001, ID : 1m4d6dq, 리뷰 : ㅜㅜ 방문했는데 아직 안열었네요ㅠㅠ 맛보고싶었는데ㅜ, 평점 : 4, 날짜 : 2021.07.22.
리뷰코드 : GC000003, 가게코드 : STORE_GC00001, ID : ssnvg6, 리뷰 : 주변 상인들 말로는 6월달부터 영업하신다고 전달받았는데 아직까지도 영업하지 않으심.. 5월 중순부터 3번은 찾아가본 듯 한데 언제쯤 먹어볼 수 있을지…, 평점 : 3, 날짜 : 2021.06.16.
리뷰코드 : GC000004, 가게코드 : STORE_GC00001, ID : 8uv76b, 리뷰 : 못먹어봐서 ㅠ 6월10일 어제 갔는데 개인 사정으로 당분간 쉰다고 적혀 있었어요 ㅠ 언제까지 쉬는지는 잘 모름 안적혀 있었음., 평점 : 3, 날짜 : 2021.06.11.
리뷰코드 : GC000005, 가게코드 : STORE_GC00001, ID : n1gc3v, 리뷰 : 만두로는 최고, 평점 : 5, 날짜 : 2021.05.19.
리뷰코드 : GC000006, 가게코드 : STORE_GC00001, ID : 11b08qg, 리뷰 : 금천구 맛집 찾다가 들렸는데 김치만두 최고네요~~ 속도 꽉차고 3개 먹으니 너무 배불르네요, 평점 : 5, 날짜 : 2020.11.27.
리뷰코드 : GC000007, 가게코드 : STORE_GC00001, ID : 1bvq504, 리뷰 : None, 평점 : 5, 날짜 : 2020.10.22.
리뷰코드 : GC000008, 가게코드 : STORE_GC00001

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=92.0.4515.131)
Stacktrace:
Backtrace:
	Ordinal0 [0x00293733+2504499]
	Ordinal0 [0x0022C401+2081793]
	Ordinal0 [0x00132628+1058344]
	Ordinal0 [0x0011BF86+966534]
	Ordinal0 [0x00177B59+1342297]
	Ordinal0 [0x00184392+1393554]
	Ordinal0 [0x00175D4B+1334603]
	Ordinal0 [0x001522B4+1188532]
	Ordinal0 [0x00153149+1192265]
	GetHandleVerifier [0x0040FB8C+1512252]
	GetHandleVerifier [0x004BB0DF+2214031]
	GetHandleVerifier [0x00314BC3+484211]
	GetHandleVerifier [0x00313E69+480793]
	Ordinal0 [0x0023218D+2105741]
	Ordinal0 [0x002366E8+2123496]
	Ordinal0 [0x00236827+2123815]
	Ordinal0 [0x0023FB73+2161523]
	BaseThreadInitThunk [0x7643FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77C67A7E+286]
	RtlGetAppContainerNamedObjectPath [0x77C67A4E+238]
	(No symbol) [0x00000000]


In [35]:
rating_df

,review_code,ItemID,UserID,review,Rating,Timestamp
0,GC000001,호세가,None,None,None,2021.06.14.
1,GC000002,호세가,None,None,None,2021.06.04.
2,GC000003,호세가,None,None,None,2021.06.02.
3,GC000004,호세가,None,None,None,2021.05.18.
4,GC000005,호세가,None,None,None,2021.05.15.
...,...,...,...,...,...,...
3229,GC003230,용호동낙지 가산점,None,None,None,2021.01.08.
3230,GC003231,용호동낙지 가산점,None,None,None,2020.01.31.
3231,GC003232,용호동낙지 가산점,None,None,None,2018.12.16.
3232,GC003233,용호동낙지 가산점,None,None,None,2018.08.29.
